## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,42.91,87,75,26.46,shower rain
1,1,Busselton,AU,-33.6500,115.3333,64.69,65,0,4.29,clear sky
2,2,Temerloh,MY,3.4483,102.4176,82.04,84,100,0.98,overcast clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,71.73,90,100,6.38,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,76.24,75,32,16.71,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Temerloh,MY,3.4483,102.4176,82.04,84,100,0.98,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,76.24,75,32,16.71,scattered clouds
13,13,Tezu,IN,27.9167,96.1667,85.28,46,7,2.57,clear sky
23,23,Faanui,PF,-16.4833,-151.7500,82.13,74,12,11.16,few clouds
24,24,Utinga,BR,-5.8172,-35.3954,77.85,88,0,9.22,light rain
30,30,Atuona,PF,-9.8000,-139.0333,78.82,76,1,14.72,clear sky
31,31,Hambantota,LK,6.1241,81.1185,82.74,94,75,10.36,broken clouds
35,35,Butaritari,KI,3.0707,172.7902,81.68,76,13,19.30,few clouds
36,36,Georgetown,MY,5.4112,100.3354,84.13,87,20,3.44,moderate rain
39,39,Kapaa,US,22.0752,-159.3190,82.38,83,14,5.99,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                188
City                   188
Country                188
Lat                    188
Lng                    188
Max Temp               188
Humidity               188
Cloudiness             188
Wind Speed             188
Current Description    188
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Temerloh,MY,82.04,overcast clouds,3.4483,102.4176,
4,Rikitea,PF,76.24,scattered clouds,-23.1203,-134.9692,
13,Tezu,IN,85.28,clear sky,27.9167,96.1667,
23,Faanui,PF,82.13,few clouds,-16.4833,-151.7500,
24,Utinga,BR,77.85,light rain,-5.8172,-35.3954,
30,Atuona,PF,78.82,clear sky,-9.8000,-139.0333,
31,Hambantota,LK,82.74,broken clouds,6.1241,81.1185,
35,Butaritari,KI,81.68,few clouds,3.0707,172.7902,
36,Georgetown,MY,84.13,moderate rain,5.4112,100.3354,
39,Kapaa,US,82.38,few clouds,22.0752,-159.3190,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))